# Notebook #5: Federated Evaluations

### Install the Rhino Health Python SDK, Load All Necessary Libraries and Login to the Rhino FCP

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.axes
import matplotlib.figure
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import getpass
import json
import io
import base64

import rhino_health as rh
from rhino_health.lib.metrics import RocAuc, RocAucWithCI

In [ ]:
my_username = "FCP_LOGIN_EMAIL" # Replace this with the email you use to log into Rhino Health
session = rh.login(username=my_username, password=getpass.getpass())

### Load the Results Cohorts from the Pneumonia Training & Validation Process

In [ ]:
results_cohorts = project.search_for_cohorts_by_name('Pneumonia training results', name_filter_mode=NameFilterMode.CONTAINS)
[cohort.name for cohort in results_cohorts]

### Calculate ROC (Underlying Results Data Stays On-prem)

In [ ]:
metric_configuration = RocAuc(y_true_variable="Pneumonia", y_pred_variable="Model Score")
results = cohort.get_metric(metric_configuration)

### Plot the ROC with your Favorite Plotting Tool

In [ ]:
roc_metrics = results.output

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

fig.canvas.draw()
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC.png", format='png', optimize=True, quality=100)

### Calculate and plot ROC with a Confidence Interval

In [ ]:
metric_configuration = RocAucWithCI(
   timeout_seconds = 30.0,
    y_true_variable="Pneumonia",
    y_pred_variable="Model Score",
    confidence_interval=95
)
results = cohort.get_metric(metric_configuration)

In [ ]:
roc_metrics = results.output

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
tpr_ci = roc_metrics['tpr_ci']
ax.fill_between(roc_metrics['fpr'], tpr_ci[0], tpr_ci[1], alpha=0.33, label='_nolegend_', color=color)
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC with Confidence Interval')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
fig.canvas.draw()
image_to_store = Image.frombytes('RGB', 
fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC_CI.png", format='png', optimize=True, quality=100)

### Upload the visualizations to the Rhino Health Platform

In [ ]:
model_result_uid = "XXXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX" # Paste the UID of the model results object for your NVF

In [ ]:
print("Sending visualizations to the Cloud")

def add_images_to_report(report_data, image_files):
    for image_file in image_files:
        with open(image_file, "rb") as temp_image:
            base_64_image = base64.b64encode(temp_image.read()).decode("utf-8")
            report_data.append(
              {
                 "type": "Image",
                 "data": {
                     "image_filename": image_file,
                     "image_base64": base_64_image,
                 },
                 "width": 100 / len(image_files)
              }
           )

roc_image_files = ('Overall_ROC.png', 'Overall_ROC_CI.png')


report_data = []
report_data.append({"type": "Title", "data": "Overall ROC"})
add_images_to_report(report_data, roc_image_files)

    

result = session.post(f"federatedmodelactions/{model_result_uid}/set_report/", 
                      data={"report_data": json.dumps(report_data)})
print('Done')